In [1]:
import pandas as pd
import numpy as np
import folium
from ipyleaflet import Map, Polyline

import warnings
warnings.simplefilter("ignore", UserWarning)

from features.map import *

In [2]:
# Load the traffic data CSV file into a pandas DataFrame
df_traffic = pd.read_csv('data/data/E-tmja-2019.csv', sep=";")
df_traffic

,dateReferentiel,route,longueur,prD,depPrD,concessionPrD,absD,cumulD,xD,yD,zD,prF,depPrF,concessionPrF,absF,cumulF,xF,yF,zF,anneeMesureTrafic,typeComptageTrafic,typeComptageTrafic_lib,TMJA,ratio_PL
0,01/01/2019,31D0044,44,0,31,N,0,0,"511656,78","6204078,36",0,0,31,N,44,44,"511698,51","6204063,93",0,NaN,NaN,NaN,NaN,NaN
1,01/01/2019,31D0044E,762,0,31,N,0,0,"511367,22","6204210,06",0,0,31,N,762,762,"511880,76","6203732,4",0,NaN,NaN,NaN,NaN,NaN
2,01/01/2019,69D0301,6055,2,69,N,-981,0,"844036,84","6510806,45",0,7,69,N,71,6055,"849346,89","6509342,34",0,NaN,NaN,NaN,NaN,NaN
3,01/01/2019,69D0383,13752,4,69,N,-494,0,"846831,3","6522372,16",0,17,69,N,544,13752,"843537,87","6512429,8",0,NaN,NaN,NaN,NaN,NaN
4,01/01/2019,69D0383BPNL,2408,0,69,N,0,0,"844982,46","6522772,27",0,2,69,N,413,2408,"846831,3","6522372,16",0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4690,01/01/2019,N1338,2060,8,76,N,0,0,"560104,39","6927892,82",0,9,76,N,960,2060,"559383,28","6929517,35",0,0.0,NaN,NaN,NaN,NaN
4691,01/01/2019,P0224,8995,211,31,N,-638,0,"555988,61","6288880,3",0,219,31,N,344,8995,"562812,02","6287175,9",0,NaN,NaN,NaN,NaN,NaN
4692,01/01/2019,P0524,1777,31,33,N,-768,0,"440252,1","6361052,92",0,31,33,N,1009,1777,"440343,36","6359374,43",0,NaN,NaN,NaN,NaN,NaN
4693,01/01/2019,P0524,3860,31,33,N,1009,1777,"458798,85","6325912,32",0,76,40,N,568,5637,"458785,94","6322872,51",0,NaN,NaN,NaN,NaN,NaN


In [3]:
df_traffic.columns

Index(['dateReferentiel', 'route', 'longueur', 'prD', 'depPrD',
       'concessionPrD', 'absD', 'cumulD', 'xD', 'yD', 'zD', 'prF', 'depPrF',
       'concessionPrF', 'absF', 'cumulF', 'xF', 'yF', 'zF',
       'anneeMesureTrafic', 'typeComptageTrafic', 'typeComptageTrafic_lib',
       'TMJA', 'ratio_PL'],
      dtype='object')

In [16]:
df_traffic['longueur'] = df_traffic['longueur'].str.replace(',', '.')

In [17]:
data = add_lat_lon_columns(df_traffic)
data

C:\Users\chach\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\chach\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\chach\OneDrive - HE

,dateReferentiel,route,longueur,prD,depPrD,concessionPrD,absD,cumulD,xD,yD,zD,prF,depPrF,concessionPrF,absF,cumulF,xF,yF,zF,anneeMesureTrafic,typeComptageTrafic,typeComptageTrafic_lib,TMJA,ratio_PL,lonD,latD,lonF,latF,region
0,01/01/2019,31D0044,44,0,31,N,0,0,511656.78,6204078.36,0,0,31,N,44,44,511698.51,6204063.93,0,NaN,NaN,NaN,NaN,NaN,0.695459,42.911382,0.695975,42.911263,Occitanie
1,01/01/2019,31D0044E,762,0,31,N,0,0,511367.22,6204210.06,0,0,31,N,762,762,511880.76,6203732.4,0,NaN,NaN,NaN,NaN,NaN,0.691871,42.912490,0.698322,42.908331,Occitanie
2,01/01/2019,69D0301,6055,2,69,N,-981,0,844036.84,6510806.45,0,7,69,N,71,6055,849346.89,6509342.34,0,NaN,NaN,NaN,NaN,NaN,4.850575,45.681629,4.918315,45.667307,Auvergne-Rhône-Alpes
3,01/01/2019,69D0383,13752,4,69,N,-494,0,846831.3,6522372.16,0,17,69,N,544,13752,843537.87,6512429.8,0,NaN,NaN,NaN,NaN,NaN,4.890020,45.785151,4.844653,45.696348,Auvergne-Rhône-Alpes
4,01/01/2019,69D0383BPNL,2408,0,69,N,0,0,844982.46,6522772.27,0,2,69,N,413,2408,846831.3,6522372.16,0,NaN,NaN,NaN,NaN,NaN,4.866352,45.789149,4.890020,45.785151,Auvergne-Rhône-Alpes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4690,01/01/2019,N1338,2060,8,76,N,0,0,560104.39,6927892.82,0,9,76,N,960,2060,559383.28,6929517.35,0,0.0,NaN,NaN,NaN,NaN,1.071779,49.435101,1.061294,49.449539,Normandie
4691,01/01/2019,P0224,8995,211,31,N,-638,0,555988.61,6288880.3,0,219,31,N,344,8995,562812.02,6287175.9,0,NaN,NaN,NaN,NaN,NaN,1.214224,43.684301,1.299264,43.670324,Occitanie
4692,01/01/2019,P0524,1777,31,33,N,-768,0,440252.1,6361052.92,0,31,33,N,1009,1777,440343.36,6359374.43,0,NaN,NaN,NaN,NaN,NaN,-0.256575,44.300286,-0.254565,44.285224,Nouvelle-Aquitaine
4693,01/01/2019,P0524,3860,31,33,N,1009,1777,458798.85,6325912.32,0,76,40,N,568,5637,458785.94,6322872.51,0,NaN,NaN,NaN,NaN,NaN,-0.007509,43.990867,-0.006227,43.963525,Nouvelle-Aquitaine


In [5]:
data['dateReferentiel'].min()

'01/01/2019'

In [6]:
data['dateReferentiel'].max()

'01/01/2019'

In [7]:
len(data['route'].unique())

323

In [8]:
# create a map centered on France
m = Map(center=(46.5, 2), zoom=6)

# create a polyline for each road based on its start and end coordinates
for idx, row in data.iterrows():
    # get the start and end latitude/longitude coordinates
    start_lat, start_lon = row['latD'], row['lonD']
    end_lat, end_lon = row['latF'], row['lonF']
    
    # create a polyline and add it to the map
    line = Polyline(locations=[(start_lat, start_lon), (end_lat, end_lon)], color='red', weight=3)
    m.add_layer(line)

# display the map
m


Map(center=[46.5, 2], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_te…

### Analyse par region

In [18]:
data_r = add_region_column(data)
data_r.head()

,dateReferentiel,route,longueur,prD,depPrD,concessionPrD,absD,cumulD,xD,yD,zD,prF,depPrF,concessionPrF,absF,cumulF,xF,yF,zF,anneeMesureTrafic,typeComptageTrafic,typeComptageTrafic_lib,TMJA,ratio_PL,lonD,latD,lonF,latF,region
0,01/01/2019,31D0044,44,0,31,N,0,0,511656.78,6204078.36,0,0,31,N,44,44,511698.51,6204063.93,0,NaN,NaN,NaN,NaN,NaN,0.695459,42.911382,0.695975,42.911263,Occitanie
1,01/01/2019,31D0044E,762,0,31,N,0,0,511367.22,6204210.06,0,0,31,N,762,762,511880.76,6203732.4,0,NaN,NaN,NaN,NaN,NaN,0.691871,42.912490,0.698322,42.908331,Occitanie
2,01/01/2019,69D0301,6055,2,69,N,-981,0,844036.84,6510806.45,0,7,69,N,71,6055,849346.89,6509342.34,0,NaN,NaN,NaN,NaN,NaN,4.850575,45.681629,4.918315,45.667307,Auvergne-Rhône-Alpes
3,01/01/2019,69D0383,13752,4,69,N,-494,0,846831.3,6522372.16,0,17,69,N,544,13752,843537.87,6512429.8,0,NaN,NaN,NaN,NaN,NaN,4.890020,45.785151,4.844653,45.696348,Auvergne-Rhône-Alpes
4,01/01/2019,69D0383BPNL,2408,0,69,N,0,0,844982.46,6522772.27,0,2,69,N,413,2408,846831.3,6522372.16,0,NaN,NaN,NaN,NaN,NaN,4.866352,45.789149,4.890020,45.785151,Auvergne-Rhône-Alpes


In [19]:
traffic_data = data_r[['region', 'longueur', 'TMJA']].fillna(0.0)
region_traffic = traffic_data.groupby('region').apply(lambda x: (x['longueur'].astype(float) * x['TMJA'].astype(float)).sum() / x['longueur'].astype(float).sum())


In [20]:
region_traffic

region
0.0                           14368.944639
Auvergne-Rhône-Alpes          28607.453593
Bourgogne-Franche-Comté       22362.332635
Bretagne                      21645.075487
Centre-Val de Loire           23155.987609
Grand Est                     20392.615743
Hauts-de-France               12935.575489
Normandie                     20335.006801
Nouvelle-Aquitaine            19296.862013
Occitanie                     24754.473324
Pays de la Loire              20020.010263
Provence-Alpes-Côte d'Azur    42976.073713
Île-de-France                 50864.868939
dtype: float64

In [21]:
import geopandas as gpd

regions = gpd.read_file('regions_shapefile.shp')

DriverError: regions_shapefile.shp: No such file or directory